# Overview:
## This Notebook contains utility functions that manage logging functionalities and provides metadata related to the tables used in the data pipeline.

In [0]:
import os 
import logging 
from datetime import datetime 
from watchtower import CloudWatchLogHandler
def createLogger(logger_name=None, log_stream=None):
    """
    Create a logger that sends logs to AWS CloudWatch.

    Parameters:
        logger_name (str): The name of the logger. Defaults to 'ETL-PipeLine' if None.
        log_stream (str): The name of the log stream in CloudWatch. Defaults to a timestamped stream name if None.

    Returns:
        logging.Logger: Configured logger instance.
    """
    # Set up logging to CloudWatch
    logger_name = logger_name or 'ETL-PipeLine'
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.DEBUG)  # Set the logger level to DEBUG

    # Define the log formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    log_group = "etl-workflow-logs"
    log_stream = log_stream or f"{logger_name}/{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"


    # Check if handlers are already added to avoid duplication
    if not logger.hasHandlers():
        # Create a CloudWatchLogHandler
        handler = CloudWatchLogHandler(log_group=log_group,log_stream_name=log_stream)
        handler.setFormatter(formatter)

        # Add the handler to the logger
        logger.addHandler(handler)
    
    return logger

def get_source_meta_data():
    """
    Provides table definitions, table names, and S3 URIs for data ingestion.

    This function returns:
    - table_definitions: A list of SQL statements defining the structure of tables in the database.
    - table_names: A tuple of names corresponding to the tables defined in `table_definitions`.
    - s3_uris: A tuple of S3 URIs for the source data files to be loaded into the corresponding tables.

      Returns:
        tuple: (table_definitions, table_names, s3_uris)
    """
    table_definitions = [
        """CREATE TABLE IF NOT EXISTS categories(
            category_id INTEGER , 
            category_name VARCHAR(150),
            sub_category_id INTEGER PRIMARY KEY, 
            sub_category_name VARCHAR(255)
        );""",
        """CREATE TABLE IF NOT EXISTS supplier(
            supplier_id INTEGER PRIMARY KEY, 
            supplier_name VARCHAR(150),
            email VARCHAR(255)
        );""",
        """CREATE TABLE IF NOT EXISTS customers(
            customer_id INTEGER PRIMARY KEY, 
            first_name VARCHAR(255) , 
            last_name VARCHAR(255) , 
            email VARCHAR(255), 
            country VARCHAR(255)
        );""",
        """
        CREATE TABLE IF NOT EXISTS payment_methods(
            payment_method_id INTEGER PRIMARY KEY, 
            payment_method VARCHAR(255)
        );""" , 
        """CREATE TABLE IF NOT EXISTS products(
            product_id INTEGER PRIMARY KEY, 
            name VARCHAR(150),
            price DECIMAL(10,2) , 
            description VARCHAR(150),
            subcategory_id INTEGER , 
            FOREIGN KEY(subcategory_id) REFERENCES categories(sub_category_id)
        );""" , 
        """
        CREATE TABLE IF NOT EXISTS orders(
            order_id_surrogate INTEGER , 
            order_id INTEGER PRIMARY KEY, 
            customer_id INTEGER , 
            order_date DATE, 
            campaign_id INTEGER ,
            amount DECIMAL(10,4) , 
            payment_method_id INTEGER ,
            FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY(payment_method_id) REFERENCES payment_methods(payment_method_id)
        );
        """ , 
        """
        CREATE TABLE IF NOT EXISTS order_items(
            orderitem_id INTEGER PRIMARY KEY, 
            order_id INTEGER , 
            product_id INTEGER , 
            quantity INTEGER, 
            supplier_id INTEGER ,
            subtotal DECIMAL(10,4) , 
            discount DECIMAL(10,4) ,
            FOREIGN KEY(order_id) REFERENCES orders(order_id),
            FOREIGN KEY(product_id) REFERENCES products(product_id),
            FOREIGN KEY(supplier_id) REFERENCES supplier(supplier_id)
        );
        """ , 
        """CREATE TABLE IF NOT EXISTS customers_ratings(
            customerproductrating_id INTEGER PRIMARY KEY, 
            customer_id INTEGER , 
            product_id INTEGER , 
            ratings DECIMAL(3,2) , 
            review VARCHAR(150) , 
            sentiment VARCHAR(150) ,
            FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
            FOREIGN KEY(product_id) REFERENCES products(product_id)
        );""" ,
        """
        CREATE TABLE IF NOT EXISTS returned_products(
            return_id INTEGER , 
            order_id INTEGER , 
            product_id INTEGER , 
            return_date DATE,
            reason VARCHAR(150),
            amount_refunded NUMERIC(10,2) , 
            FOREIGN KEY(order_id) REFERENCES orders(order_id),
            FOREIGN KEY(product_id) REFERENCES products(product_id)
        );""" 
        ] 
    
    table_names = ('categories','supplier','customers','payment_methods','products',
               'orders','order_items','customers_ratings','returned_products') 
    s3_base_uri = "Datasource/"
    s3_file_keys = (
        'categoryDenormalized.csv' , 'supplier.csv','customer.csv' , 
        'payment_method.csv', 'product.csv', 'orders.csv','orderitem.csv',
        'customer_product_ratings.csv' ,'returns.csv'
    )
    s3_resource_path = tuple(s3_base_uri + uri for uri in s3_file_keys )
    return table_definitions , table_names , s3_resource_path


In [0]:
from functools import wraps
import traceback
import json
import time
from datetime import datetime , timedelta 

class Logger:
    """
    Custom Logger class that provides a decorator for logging metadata about function executions.

    This logger captures details such as:
    - Function name
    - Execution status (Success/Failed)
    - Function docstring
    - Start and end time of execution
    - Total execution time
    - Any errors or exceptions that occur during execution

    The logs are stored in a dictionary with a unique identifier for each function call, 
    allowing for easy tracking and debugging of function behavior.
    """
    def __init__(self):
        self.unique_id = 0
        self.logs = {}
        self.log_structure = {
            "function_name": "",
            "status":"Success" , 
            "doc_string": "",
            "start_time": "",
            "end_time": "",
            "execution_time": "",
            "error": "Not occurred",
            "exception": "Not occurred",
        }

    def logger(self, func):
        """Decorator function to log details of the wrapped function execution."""
        @wraps(func) # Preserve the wrapped function's metadata
        def decorator(*args, **kwargs):

            # Record the start time of the function execution
            start = datetime.now() + timedelta(hours=5,minutes=30)

            # Create a new log entry with a fresh copy of the log structure
            self.logs[self.unique_id] = self.log_structure.copy()
            try:
                # Execute the wrapped function
                result = func(*args, **kwargs)
            except Exception as e:
                cache_exception = f"{traceback.format_exc()}"
                self.logs[self.unique_id].update(
                    error=str(e), exception=cache_exception ,
                    status = "Failed"

                )
                raise  # Re-raise the error for further handling
            finally:
                # Record the end time of the function execution
                end = datetime.now() + timedelta(hours=5, minutes=30)
                # Calculate the execution time
                execution_time = end - start
                # Update the log entry with execution details
                self.logs[self.unique_id].update(
                    function_name=func.__name__ or " ",
                    doc_string=json.dumps(func.__doc__, indent=4) if func.__doc__ else "No Docstring Available",
                    start_time=start.strftime("%Y-%m-%d %H:%M:%S"),
                    end_time=end.strftime("%Y-%m-%d %H:%M:%S"),
                    execution_time=f"{execution_time.total_seconds()} seconds",
                )
                # Increment the unique ID for the next log entry
                self.unique_id +=1 
            return result
        return decorator
    def clear_logs(self):
        """ Delete Existing Logs """ 
        self.logs.clear() 


# f = Logger()


# @f.log
# def add():
#     """This function adds two numbers."""
#     time.sleep(2) 
#     return 2 /0 


# @f.log
# def add1():
#     """This function adds another set of numbers."""
#     time.sleep(2) 
# try:
#     add() 
#     add1() 
# except Exception as ex:
#     print(ex) 

# print(json.dumps(f.logs,indent=4))
